In [ ]:
import os 
from dotenv import load_dotenv
load_dotenv()

groq_api_key= os.getenv("GROQ_API_KEY")
groq_api_key

In [ ]:
!pip install langchain_groq

In [ ]:
from langchain_groq import ChatGroq
model = ChatGroq(model='Gemma2-9b-It',api_key=groq_api_key)
model

In [ ]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, I am Pradum and I am a Data Scientist.")])


In [ ]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, I am Pradum and I am a Data Scientist."),
        AIMessage(content="Hello Pradum! It's nice to meet you. \n\nAs a large language model, I'm always interested in learning more about people in different fields."),
        HumanMessage(content="What is my name and what do i do?")

    ]
)

# Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!


In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [ ]:
config = {'configurable':{"session_id":'chat1'}}

In [ ]:
response= with_message_history.invoke(
    [HumanMessage(content='Hi, my name is Pradum and I am Data Scientist.')],
    config=config
)

In [ ]:
response.content

In [ ]:
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)

In [ ]:
# Change the config --> session id
config1 = {'configurable':{'session_id':'chat2'}}
response = with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config1
)

In [ ]:
response.content

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content='Hey my name is Jhon')],
    config = config1
)
response.content

In [ ]:
with_message_history.invoke([
    HumanMessage(content="what's my name")],
    config=config1
)

# Prompt Template
it helps us to turn raw user information into a format that the llm can work with

In [ ]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",'you are a helpful assistant. Answer all the question to the nest of your ability.'),
        MessagesPlaceholder(variable_name="messages")

    ]
)
chain = prompt|model

In [ ]:
chain.invoke({'messages':[HumanMessage(content='Hi My name us Pradum')]})

In [ ]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history=get_session_history)

In [ ]:
config = {'configurable':{'session_id':'chat3'}}
response = with_message_history.invoke(
    [HumanMessage(content='Hi my name is Pradum')],
    config=config
)
response

In [ ]:
with_message_history.invoke(
    [HumanMessage(content='What is my name')],
    config=config
)

In [ ]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            "You are a helpful assistant. Answer all the question to the best of your ability in {language}.",

        ),
        MessagesPlaceholder(variable_name='messages'),
    ]
)
chain = prompt|model

In [ ]:
chain.invoke({'messages':[HumanMessage(content='Hi my name is Pradum')],'language':'Hindi'})

Let's wrap this more complicated chain in a message history class. this time, because there are multiple keys in the input, we need to specify the correct key to save the hcat history

In [ ]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key='messages'
    
)

In [ ]:
config = {'configurable':{'session_id':'chat4'}}
response = with_message_history.invoke(
    {'messages':[HumanMessage(content="Hi I am Pradum")],'language':'Hindi'},
    config=config
)

In [ ]:
response.content

In [ ]:
response = with_message_history.invoke({'messages':[HumanMessage(content='What is my name?')],'language':'Hindi'},
                                       config=config
                                       )


In [ ]:
response.content

# Managing the conversation History


In [ ]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer = trim_messages(
    max_tokens =45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on='human'

)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter('messages')|trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content='what ice cream do i like')],
        "language":"English"
    }
)
response.content

In [ ]:
response = chain.invoke({"messages":messages + [HumanMessage(content='What math problem is gave')],"language":"English"})
response.content

In [ ]:
# Lets wrap this is message history.

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config = {"configurable":{"session_id":"chat5"}}

In [ ]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="What's is my name")],
        "language":"english"
    },
    config= config,
)
response.content

In [ ]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="What problen did i ask")],
        "language":"english"
    },
    config= config,
)
response.content